# Filter the glacier length and area time series

In this short tutorial, we show how to deal with unwanted "spikes" in the length and area time series of individual glaciers. These happen because OGGM currently doesn't differentiate between snow and ice, i.e. occasional years with large snowfall can artificially increase the glacier area.

While the best solution would be to deal with this in OGGM, this is currently not possible because we do not have a generally applicable solution to this problem. In the meantime, we recommend a simple workaround.

## Set-up 

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import os

In [ ]:
from oggm import cfg, utils, workflow, tasks
cfg.initialize()

In [ ]:
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-Filter')

### Define the glaciers for the run 

We take the Kesselwandferner in the Austrian Alps:

In [ ]:
rgi_ids = ['RGI60-11.00787']

### Glacier directories 

In [ ]:
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4, prepro_border=80)

## Run

We can step directly to a new experiment! This runs under a random climate representative for the recent climate (1985-2015) and a warm temperature bias:

In [ ]:
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=200, y0=2000, seed=5,
                             output_filesuffix='_commitment');

## The problem 

In [ ]:
ds = utils.compile_run_output(gdirs, filesuffix='_commitment')
ds = ds.isel(rgi_id=0)  # take just the one glacier

In [ ]:
ds.area.plot();

In [ ]:
ds.length.plot();

For small areas, the glacier has the unrealistic "spikes" described above.

## Workaround 

A good way to deal with the issue is to run a moving filter which keeps the smallest area or length in a given window size:

In [ ]:
roll_yrs = 5

In [ ]:
# Take the minimum out of 5 years
ts = ds.area.to_series()
ts = ts.rolling(roll_yrs).min()
ts.iloc[0:roll_yrs] = ts.iloc[roll_yrs]

In [ ]:
# Plot
ds.area.plot(label='Original');
ts.plot(label='Filtered');
plt.legend();

It works the same with length:

In [ ]:
# Take the minimum out of 5 years
ts = ds.length.to_series()
ts = ts.rolling(roll_yrs).min()
ts.iloc[0:roll_yrs] = ts.iloc[roll_yrs]
# Plot
ds.length.plot(label='Original');
ts.plot(label='Filtered');
plt.legend();

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)